## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

And to obtain integer representation, the real value $x_i$

$$ x_i = \sum_{l=-m}^m 2^l q_{i,l}^+ - \sum_{l=-m}^m 2^l q_{i,l}^- $$

To present both position and negetive numbers $q_{i,l}^+$ and $q_{i,l}^-$ are involved.

As before, the domains are given by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q_r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 2
qubits = 2
A = np.array([[4, -2], [-5, -5]])
b = np.array([20, -10])

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
# print("QM" + str(QM));
for k in range(Dimension): 
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 4*i + l
            po2 = 4*i + l + 2
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2

# print("after calculating linear terms: ")
# print(QM)

for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 4*i + l1
                po2 = 4*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 4*i + l1 + 2
                po4 = 4*i + l2 + 2
                QM[po3][po4] = QM[po3][po4] + qcef

# print("after calculating quadratic terms: ")
# print(QM)

for k in range(Dimension):
    for i in range(Dimension-1):
            for j in range(i+1,Dimension):
                for l1 in range(qubits):
                    for l2 in range(qubits):
                        qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j]
                        po1 = 4*i + l1
                        po2 = 4*j + l2
                        QM[po1][po2] = QM[po1][po2] + qcef
                        po3 = 4*i + l1 + 2
                        po4 = 4*j + l2 + 2
                        QM[po3][po4] = QM[po3][po4] + qcef
                        po5 = 4*i + l1
                        po6 = 4*j + l2 + 2
                        QM[po5][po6] = QM[po5][po6] - qcef
                        po7 = 4*i + l1 + 2
                        po8 = 4*j + l2
                        QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)

# Matrix Q is
[[-219.  164.    0.    0.   34.   68.  -34.  -68.]
 [   0. -356.    0.    0.   68.  136.  -68. -136.]
 [   0.    0.  301.  164.  -34.  -68.   34.   68.]
 [   0.    0.    0.  684.  -68. -136.   68.  136.]
 [   0.    0.    0.    0.    9.  116.    0.    0.]
 [   0.    0.    0.    0.    0.   76.    0.    0.]
 [   0.    0.    0.    0.    0.    0.   49.  116.]
 [   0.    0.    0.    0.    0.    0.    0.  156.]]


## Apply QUBO

In [96]:
qbit_list = []
for val in range(len(QM)):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)
Q2 = {}
for i in range(len(QM)):
    for j in range(len(QM)):
        if QM[i][j] != 0:
            Q2[(qbit_list[i],qbit_list[j])] = QM[i][j]
            
print(Q2)
            

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8']
{('q1', 'q1'): -219.0, ('q1', 'q2'): 164.0, ('q1', 'q5'): 34.0, ('q1', 'q6'): 68.0, ('q1', 'q7'): -34.0, ('q1', 'q8'): -68.0, ('q2', 'q2'): -356.0, ('q2', 'q5'): 68.0, ('q2', 'q6'): 136.0, ('q2', 'q7'): -68.0, ('q2', 'q8'): -136.0, ('q3', 'q3'): 301.0, ('q3', 'q4'): 164.0, ('q3', 'q5'): -34.0, ('q3', 'q6'): -68.0, ('q3', 'q7'): 34.0, ('q3', 'q8'): 68.0, ('q4', 'q4'): 684.0, ('q4', 'q5'): -68.0, ('q4', 'q6'): -136.0, ('q4', 'q7'): 68.0, ('q4', 'q8'): 136.0, ('q5', 'q5'): 9.0, ('q5', 'q6'): 116.0, ('q6', 'q6'): 76.0, ('q7', 'q7'): 49.0, ('q7', 'q8'): 116.0, ('q8', 'q8'): 156.0}


In [97]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

# print(Q)

sampleset = sampler_auto.sample_qubo(Q2, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0  1  1  0  0  0  0  1  0 -464.0     382     0.0
1  1  1  0  0  0  0  0  1 -459.0     605     0.0
2  1  1  0  0  0  0  0  0 -411.0       2     0.0
3  1  1  0  0  0  0  1  1 -396.0       7     0.0
4  0  1  0  0  0  0  1  0 -375.0       2     0.0
5  1  1  0  0  1  0  1  0 -353.0       1     0.0
6  1  1  0  0  1  0  0  1 -348.0       1     0.0
['BINARY', 7 rows, 1000 samples, 8 variables]


### Convert Qbit to Decimal 

In [98]:
samples = sampleset.samples()
type(samples[0])
qbit_per_var_count = int(len(qbit_list)/Dimension)
print(qbit_per_var_count)

qbit_per_var = []
for i in range(Dimension):
    qbit_per_var.append(samples[i,qbit_list[qbit_per_var_count*i:qbit_per_var_count*(i+1)]])

print(qbit_per_var)
res = []

for i in range(qubits):
    res.append(0)
    for j in range(qubits):
        res[i] += (pow(2,j)*qbit_per_var[i][j] -  pow(2,j)*qbit_per_var[i][qubits+j])

print("Output: ", res)
    


4
[array([1, 1, 0, 0], dtype=int8), array([0, 0, 0, 1], dtype=int8)]
Output:  [3, -2]


In [94]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q2)
print(J)

({'q1': -68.5, 'q2': -137.0, 'q5': 33.5, 'q6': 67.0, 'q7': 53.5, 'q8': 107.0, 'q3': 191.5, 'q4': 383.0}, {('q1', 'q2'): 41.0, ('q1', 'q5'): 8.5, ('q1', 'q6'): 17.0, ('q1', 'q7'): -8.5, ('q1', 'q8'): -17.0, ('q2', 'q5'): 17.0, ('q2', 'q6'): 34.0, ('q2', 'q7'): -17.0, ('q2', 'q8'): -34.0, ('q3', 'q4'): 41.0, ('q3', 'q5'): -8.5, ('q3', 'q6'): -17.0, ('q3', 'q7'): 8.5, ('q3', 'q8'): 17.0, ('q4', 'q5'): -17.0, ('q4', 'q6'): -34.0, ('q4', 'q7'): 17.0, ('q4', 'q8'): 34.0, ('q5', 'q6'): 29.0, ('q7', 'q8'): 29.0}, 490.0)


In [92]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0 -1 -1 +1 -1 -1 +1 -1 -1 -131.0     864     0.0
1 -1 -1 +1 -1 +1 -1 -1 -1 -126.0      60     0.0
2 -1 -1 +1 -1 +1 +1 -1 -1 -126.0      60     0.0
3 -1 -1 -1 +1 -1 +1 -1 -1 -121.0      13     0.0
4 -1 -1 -1 +1 +1 +1 -1 -1 -118.0       2     0.0
5 -1 -1 +1 -1 -1 -1 -1 -1 -111.0       1     0.0
['SPIN', 6 rows, 1000 samples, 8 variables]
